In [16]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
def matrix_vector_product(A, x):
    """
    Calculates matrix-vector product.
    Args:
        A (array_like): A m-by-n matrix
        x (array_like): Vector of size n
    Returns:
        ndarray: Matrix-vector product
    """
    m, n = A.shape
    b = np.zeros(m)
    for i in range(m):
        for j in range(n):
            b[i] += A[i,j] * x[j]
    return b

In [45]:
# test of the matrix_vector_product function
A = np.random.rand(3, 3)
x = np.random.rand(3)
print(matrix_vector_product(A, x))
print(np.dot(A, x))

[ 0.85582148  1.15336365  0.920908  ]
[ 0.85582148  1.15336365  0.920908  ]


In [32]:
def matrix_matrix_product(A, B):
    """
    Calculates matrix-matrix product.
    Args:
        A (array_like): A m-by-n matrix
        B (array_like): A n-by-p matrix
    Returns:
        ndarray: Matrix-matrix product
    """
    m, n = A.shape
    n, p = B.shape
    C = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            for k in range(n):
                C[i,j] += A[i,k] * B[k,j]
    return C

In [44]:
# test of the matrix_matrix_product function
A = np.random.rand(3, 3)
B = np.random.rand(3, 3)
print(matrix_matrix_product(A, B))
print(np.dot(A, B))

[[ 0.86787944  1.34268978  0.95902293]
 [ 1.31335296  2.07779107  1.43701576]
 [ 0.73857971  1.09117205  0.66714489]]
[[ 0.86787944  1.34268978  0.95902293]
 [ 1.31335296  2.07779107  1.43701576]
 [ 0.73857971  1.09117205  0.66714489]]


In [9]:
def forward_substitution(A, b):
    """
    Solves a system of linear equation with lower triangular matrix.
    Args:
        A (array_like): A n-by-n lower triangular matrix
        b (array_like): RHS vector of size n
    Returns:
        ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    for i in range(0, n):
        x[i] = (b[i] - np.dot(A[i,:], x)) / A[i,i]
    return x

In [43]:
# test of the forward_substitution function
A = np.tril(np.random.rand(3, 3))
b = np.random.rand(3)
print(forward_substitution(A, b))
print(la.solve(A, b))

[ 4.32098444 -4.31617618  7.14131761]
[ 4.32098444 -4.31617618  7.14131761]


In [10]:
def backward_substitution(A, b):
    """
    Solves a system of linear equation with upper triangular matrix.
    Args:
        A (array_like): A n-by-n upper triangular matrix
        b (array_like): RHS vector of size n
    Returns:
        ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i,:], x)) / A[i,i]
    return x

In [42]:
# test of the backward_substitution function
A = np.triu(np.random.rand(3, 3))
b = np.random.rand(3)
print(backward_substitution(A, b))
print(la.solve(A, b))

[-2.30033501  2.90840673  0.51824484]
[-2.30033501  2.90840673  0.51824484]


In [11]:
def gaussian_elimination(A, b):
    """
    Transform given matrix into upper triangular form, perform identical operations on RHS vector.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
    Returns:
        ndarray: Upper triangular matrix
        ndarray: RHS vector corresponding to upper triangular matrix
    """
    n, n = A.shape
    tmp = np.zeros((n, n + 1))
    tmp[:,:-1] = A
    tmp[:,-1] = b
    for i in range(0, n):  
        # return the index of max. row relatively to i (max_row 0 means that the i-th row has maximum)
        max_row = np.argmax(A[i:,i])
        if (max_row != 0):
            row_i = np.copy(tmp[i,:])
            tmp[i,:] = np.copy(tmp[i+max_row,:])
            tmp[i+max_row,:] = np.copy(row_i)    
        for j in range(i + 1, n):
            tmp[j,:] = tmp[j,:] - (tmp[j,i] / tmp[i,i]) * tmp[i,:]      
    return tmp[:,:-1], tmp[:,-1]

In [68]:
# test of the gaussian_elimination function
A = np.random.rand(3, 3)
b = np.random.rand(3)
A_upper, bb = gaussian_elimination(A, b)
print(backward_substitution(A_upper, bb))
print(la.solve(A, b))

[ 1.19579821 -1.18796401  1.41614536]
[ 1.19579821 -1.18796401  1.41614536]


In [12]:
def lu_decomposition(A):
    """
    Decompose given matrix into a product of a lower and an upper triangular matrix.
    Args:
        A (array_like): A n-by-n regular matrix
    Returns:
        ndarray: Lower triangular matrix
        ndarray: Upper triangular matrix
    """
    n, n = A.shape
    L = np.zeros((n, n))
    U = np.zeros((n, n))
    for i in range(0, n):
        for j in range(i, n):
            U[i,j] = A[i,j] - np.dot(L[i,:], U[:,j])
        for j in range(i, n):
            L[j,i] = (A[j,i] - np.dot(L[j,:], U[:,i])) / U[i,i]
    return L, U

In [73]:
# test of the lu_decomposition function
A = np.random.rand(3, 3)
b = np.random.rand(3)
L, U = lu_decomposition(A)
y = forward_substitution(L, b)
x = backward_substitution(U, y)
print(x)
print(la.solve(A, b))

[  7.22333086   0.75865305 -10.09360607]
[  7.22333086   0.75865305 -10.09360607]


In [13]:
def thomas_algorithm(A, f):
    """
    Solves system of linear equations with tridiagonal matrix using Thomas' algorithm.
    Args:
        A (array_like): A n-by-n regular matrix
        f (array_like): RHS vector of size n
    Returns:
        ndarray: Vector of solution
    """
    n, n = A.shape # get the size of input matrix
    c = np.diag(A, -1) # get elements below diagonal 
    a = np.diag(A, 0) # get elements on diagonal
    b = np.diag(A, 1) # get elements above diagonal
    c = np.insert(c, 0, 0.) # insert 0 as a first element of c
    b = np.insert(b, b.size, 0.) # insert 0 as a last element of b
    x = np.zeros(n + 1) 
    rho = np.zeros(n + 1)
    mu  = np.zeros(n + 1)
    for i in range(0, n):
        mu[i] = -b[i] / (c[i] * mu[i-1] + a[i])
        rho[i] = (f[i] - c[i] * rho[i-1]) / (c[i] * mu[i-1] + a[i])
    for i in range(n-1, -1, -1):
        x[i] = mu[i] * x[i+1] + rho[i]
    return x[:-1]

In [111]:
# test of the thomas_algorithm function
A = np.random.rand(5, 5)
b = np.random.rand(5)

# create tridiagonal matrix from random matrix A
for i in range(0, 3):
    for j in range(i + 2, 5):
        A[i,j] = 0.0
        
for i in range(2, 5):
    for j in range(0, i - 1):
        A[i,j] = 0.0
        
print(thomas_algorithm(A, b))
print(la.solve(A,b))

[ 0.68244088 -0.18589126 -0.02155454  0.71584894  0.60568739]
[ 0.68244088 -0.18589126 -0.02155454  0.71584894  0.60568739]


In [14]:
def jacobi_method(A, b, max_it=500, eps=0.0):
    """
    Solves system of linear equations iteratively using Jacobi's algorithm.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
        max_it (int): Maximum number of iterations
        eps (float): Error tolerance
    Returns:
        ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    x_new = np.zeros(n)   
    for k in range(max_it):
        for i in range(n):
            x_new[i] = (1.0 / A[i,i]) * (b[i] - np.dot(A[i,:i], x[:i]) - np.dot(A[i,i+1:], x[i+1:]))           
        x = x_new
        if(la.norm(np.dot(A, x) - b) < eps):
            break   
    return x

In [122]:
# test of the jacobi_method function
A = np.random.rand(3, 3) + 10. * np.eye(3) # create diagonally dominant matrix to ensure convergence
b = np.random.rand(3)
print(jacobi_method(A, b, 20))
print(la.solve(A, b))

[ 0.02672524  0.07000596  0.09159908]
[ 0.02672524  0.07000596  0.09159908]


In [15]:
def gauss_seidel_method(A, b, max_it=500, eps=0.0):
    """
    Solves system of linear equations iteratively using Gauss-Seidel's algorithm.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
        max_it (int): Maximum number of iterations
        eps (float): Error tolerance
    Returns:
        ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    x_new = np.zeros(n)  
    for k in range(max_it):
        for i in range(n):
            x_new[i] = (1.0 / A[i,i]) * (b[i] - np.dot(A[i,:i], x_new[:i]) - np.dot(A[i,i+1:], x[i+1:]))      
        x = x_new
        if(la.norm(np.dot(A, x) - b) < eps):
            break  
    return x

In [125]:
# test of the gauss_seidel_method function
A = np.random.rand(3, 3) + 10. * np.eye(3) # create diagonally dominant matrix to ensure convergence
b = np.random.rand(3)
print(gauss_seidel_method(A, b, 20))
print(la.solve(A, b))

[-0.00108194  0.03062653  0.08532184]
[-0.00108194  0.03062653  0.08532184]


In [11]:
def successive_overrelaxation_method(A, b, max_it=500, eps=0.0):
    """
    Solves system of linear equations iteratively using successive overrelaxation (SOR) method.
    Args:
        A (array_like): A n-by-n regular matrix
        b (array_like): RHS vector of size n
        max_it (int): Maximum number of iterations
        eps (float): Error tolerance
    Returns:
        ndarray: Vector of solution
    """
    n, n = A.shape
    x = np.zeros(n)
    x_new = np.zeros(n)
    L = np.tril(A, -1) # get lower triangular matrix with zeros on diagonal
    U = np.triu(A, 1) # get upper triangular matrix with zeros on diagonal
    D = A - L - U # get diagonal matrix
    B = -np.dot(la.inv(D + L), U) # calculate iteration matrix
    rho = np.max(np.abs(la.eigvals(B))) # find spectral radius (i.e. maximal eigenvalue in absolute value)
    omega = 2.0 / (1.0 + np.sqrt(np.abs(1.0 - rho**2))) # find optimal relaxation factor
    for k in range(max_it):
        for i in range(n):
            x_new[i] = (1.0 / A[i,i]) * (b[i] - np.dot(A[i,:i], x_new[:i]) - np.dot(A[i,i+1:], x[i+1:]))
        x += omega * (x_new - x)
        if(la.norm(np.dot(A, x) - b) < eps):
            break
    return x

In [12]:
# test of the successive_overrelaxation_method function
A = np.random.rand(3, 3) + 10. * np.eye(3) # create diagonally dominant matrix to ensure convergence
b = np.random.rand(3)
print(successive_overrelaxation_method(A, b, 20))
print(la.solve(A, b))

[ 0.07719775 -0.0095787   0.06860561]
[ 0.07719775 -0.0095787   0.06860561]


In [13]:
def power_iteration(A, max_it=500):
    """
    Finds the greatest eigen value (in absolute value) of given matrix and its corresponding eigenvector.
    Args:
        A (array_like): A n-by-n diagonalizable matrix
        max_it (int): Maximum number of iterations
    Returns:
        ndarray: Eigenvector corresponding to a greatest eigenvalue (in absolute value)
        float: Greatest eigenvalue (in absolute value)
    """
    n, n = A.shape
    eigen_vec = np.ones(n)
    for i in range(max_it):
        eigen_vec_new = np.dot(A, eigen_vec)
        eigen_vec = eigen_vec_new / la.norm(eigen_vec_new)
    eigen_val = la.norm(np.dot(A, eigen_vec))
    return eigen_vec, eigen_val

In [14]:
# test of the power_iteration function
A = np.random.rand(3, 3)
e_vec, e_val = power_iteration(A, 20)
print(e_val)
print(np.max(np.abs(la.eigvals(A))))

1.27801307047
1.27801866444
